# Email Spam Detection With Machine Learning
## Project by MEGHA MITTAL


### Importing Neccessary Libraries

In [21]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
from sklearn.metrics import confusion_matrix , accuracy_score , classification_report, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier


### Loading the dataset

In [22]:
df= pd.read_csv('spam.csv', encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


##### As you can see in reading the csv file,  I have given second argument named as encoding ,this encoding parameter specifies the character encoding of the file. In this case, 'ISO-8859-1' is used, which is a common encoding for text files.

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


##### The files contain one message per line. Each line is composed by two columns: v1 contains the label (ham or spam) and v2 contains the raw text.



##### As we can see in the info of df dataframe, columns named as unmanned:2, unmanned:3, unmanned:4 are irrelevant, so we have to drop them before moving towards more preprocessing steps.

In [24]:
df.shape

(5572, 5)

##### We have 5572 rows of data, which is message and its corresponding label which can be either a ham or a spam email.

In [25]:
df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)

In [26]:
df.columns = ['label', 'message']

##### Let's rename the column names to more logical names as message and label for better understanding.


In [27]:
df.columns

Index(['label', 'message'], dtype='object')

In [28]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [29]:
df.isnull().sum()

label      0
message    0
dtype: int64

#### No missing values

In [30]:
print(df.duplicated().sum())

403


##### We have 403 emails which are duplicate of each other. So, lets remove these so that our model should not get overfitted while learning the same emails which are residing in our dataset(spam.csv)

In [31]:
df= df.drop_duplicates(keep='first')
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


#####  As you can notice the argument is setted in duplicate function which is the "keep" parameter is set to 'first'. This means that when duplicate rows are found, only the first occurrence of each duplicate row will be kept, and all subsequent occurrences will be removed.

In [32]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


##### Notice in our dataset, the data is in the text format, but models or neural networks understands numbers, so we have to convert our text data into number representation of them. For this task, lets use labelencoding, or you can use other also like word2vec, embeddings in the nltk library etc.

### Label Encoding

In [35]:
encoder= LabelEncoder()
df['label']= encoder.fit_transform(df['label'])
df

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [36]:
df['label'].value_counts()

label
0    4516
1     653
Name: count, dtype: int64

#### We have 4516 ham emails and rest 653 are spam emails